## Speed comparison between fast.ai and torchvision dataloaders. Using cifar-10 dataset with WideResNet22 as model.

In [1]:
# sources:
#  https://github.com/fastai/fastai/blob/master/courses/dl2/cifar10-dawn.ipynb
#  https://github.com/fastai/imagenet-fast/blob/master/cifar10/cifar10-super-convergence-tuned.ipynb
#  https://github.com/radekosmulski/machine_learning_notebooks/blob/master/cifar10_fastai_dawnbench.ipynb

In [ ]:
# my hardware:
#    Intel i7-7700K@4.2GHz x 8(CPU) (actually 4 cores), 32GB of RAM
#    Zotac GeForce 1080Ti 11GB
#    MSI Z170I Gaming Pro motherboard

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.models.cifar10.wideresnet import wrn_22
from datetime import datetime
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10_dirs_by_classes/")

In [3]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
means,stdevs = (0.4914, 0.48216, 0.44653), (0.24703, 0.24349, 0.26159)
stats = (np.array(means), np.array(stdevs))

bs=512
sz=32
wd=1e-4
lr=1.5
nw=num_cpus()

## ver1: with fast.ai dataloaders

In [11]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs, num_workers=nw)

In [12]:
m = wrn_22()

In [13]:
learn = ConvLearner.from_model_data(m, data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]

In [7]:
start_time = datetime.now()
%time learn.fit(lr, 1, wds=wd, cycle_len=2, use_clr_beta=(20,20,0.95,0.85))
end_time = datetime.now()

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.528044   1.415433   0.491     
    1      1.071998   0.902383   0.673                    

CPU times: user 1min 9s, sys: 36.6 s, total: 1min 45s
Wall time: 1min 29s


[array([0.90238]), 0.673]

In [ ]:
str(end_time - start_time)

In [14]:
start_time = datetime.now()
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20,20,0.95,0.85))
end_time = datetime.now()

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.463027   1.828972   0.4145    
    1      1.047709   1.172208   0.6129                   
    2      0.830867   0.928634   0.6605                    
    3      0.674442   1.056057   0.6641                    
    4      0.572754   0.74402    0.7418                    
    5      0.502476   0.60076    0.794                     
    6      0.461384   0.548186   0.8144                    
    7      0.427653   0.526555   0.8237                    
    8      0.394464   0.542668   0.8191                    
    9      0.365919   0.511267   0.8277                    
    10     0.369863   0.515518   0.8269                    
    11     0.338405   0.573855   0.8167                    
    12     0.324958   0.682364   0.785                     
    13     0.296051   0.804754   0.759                     
    14     0.283267   0.663231   0.7965                    
    15     0.267195   0.594402   0.812                    

In [15]:
str(end_time - start_time)

'0:21:34.236471'

## ver2: with torchvision dataloaders

In [4]:
from torchvision import transforms as tvtf, datasets as tvds
from torch.utils.data import DataLoader as tdl

In [5]:
def get_loaders(bs, num_workers):
    traindir, valdir = str(PATH/'train'), str(PATH/'test')
    tfms = [tvtf.ToTensor(), tvtf.Normalize(means,stdevs)]
    aug_tfms = tvtf.Compose([tvtf.RandomCrop(sz, padding=4), tvtf.RandomHorizontalFlip()] + tfms)

    train_dataset = tvds.ImageFolder(traindir, aug_tfms)
    val_dataset = tvds.ImageFolder(valdir, tvtf.Compose(tfms))
    aug_dataset = tvds.ImageFolder(valdir, aug_tfms)

    train_loader = tdl(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = tdl(val_dataset, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=True)
    aug_loader = tdl(aug_dataset, batch_size=bs, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, aug_loader

def get_data(bs, num_workers):
    trn_dl, val_dl, aug_dl = get_loaders(bs, num_workers)
    d = ModelData(PATH, trn_dl, val_dl)
    d.aug_dl = aug_dl
    d.sz=sz
    return d

data = get_data(bs, nw)

In [6]:
m = wrn_22()

In [7]:
learn = ConvLearner.from_model_data(m, data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]

In [12]:
start_time = datetime.now()
%time learn.fit(lr, 1, wds=wd, cycle_len=2, use_clr_beta=(20,20,0.95,0.85))
end_time = datetime.now()

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.580392   1.649794   0.4413    
    1      1.1049     0.96543    0.6573                   

CPU times: user 54.7 s, sys: 24.4 s, total: 1min 19s
Wall time: 1min 19s


[array([0.96543]), 0.6573]

In [ ]:
str(end_time - start_time)

In [8]:
start_time = datetime.now()
%time learn.fit(lr, 1, wds=wd, cycle_len=30, use_clr_beta=(20,20,0.95,0.85))
end_time = datetime.now()

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                 
    0      1.466761   4.471314   0.2675    
    1      1.072684   1.31925    0.5971                   
    2      0.852208   0.979466   0.6731                    
    3      0.693892   0.783403   0.7326                    
    4      0.597067   0.623417   0.7849                    
    5      0.522471   0.723941   0.7585                    
    6      0.482226   0.794252   0.7452                    
    7      0.437199   0.707461   0.7717                    
    8      0.407294   0.659782   0.7833                    
    9      0.388441   0.607679   0.8039                    
    10     0.369216   0.678423   0.7856                    
    11     0.347235   1.174847   0.6896                    
    12     0.333878   0.584167   0.8052                    
    13     0.312995   0.952613   0.7358                    
    14     0.306139   0.498115   0.8396                    
    15     0.279794   0.682821   0.7882                   

In [9]:
str(end_time - start_time)

'0:21:08.322093'

## Conclusion

Average run times:


ver1 = 90sec


ver2 = 80sec
